# Principle component analysis training (MADlib v1.10+)
Demonstrates PCA training including grouping, which was introduced in MADlib v1.10

In [211]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [212]:
%sql postgresql://gpdbchina@10.194.10.68:55000/madlib
#%sql postgresql://fmcquillan@localhost:5432/madlib
#%sql postgresql://gpadmin@54.197.30.46:10432/gpadmin

u'Connected: gpdbchina@madlib'

In [213]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.10.0-dev, git revision: rc/v1.9alpha-rc1-91-ga0325d6, cmake configuration time: Wed Jan 11 01:36:53 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


# 1.  Create sample data in dense matrix form

In [284]:
%%sql 
DROP TABLE IF EXISTS mat;

CREATE TABLE mat (
    id integer,
    row_vec double precision[]
);

INSERT INTO mat VALUES
(1, '{1,2,3}'),
(2, '{2,1,2}'),
(3, '{3,2,1}');

SELECT * FROM mat ORDER BY id;

Done.
Done.
3 rows affected.
3 rows affected.


id,row_vec
1,"[1.0, 2.0, 3.0]"
2,"[2.0, 1.0, 2.0]"
3,"[3.0, 2.0, 1.0]"


# 2.  Run the PCA function for a specified number of principal components

In [285]:
%%sql
DROP TABLE IF EXISTS result_table, result_table_mean;

SELECT madlib.pca_train('mat',             -- Source table
                        'result_table',    -- Output table
                        'id',              -- Row id of source table
                         2);               -- Number of principle components

SELECT * FROM result_table ORDER BY row_id;

Done.
1 rows affected.
2 rows affected.


row_id,principal_components,std_dev,proportion
1,"[0.707106781186547, 0.0, -0.707106781186547]",1.41421356237,0.857142857142
2,"[0.0, -1.0, 0.0]",0.57735026919,0.142857142857


# 3.  Run the PCA function for a specified proportion of variance

In [286]:
%%sql
DROP TABLE IF EXISTS result_table, result_table_mean;
SELECT madlib.pca_train('mat',             -- Source table
                        'result_table',    -- Output table
                        'id',              -- Row id of source table
                         0.9);             -- Proportion of variance
SELECT * FROM result_table ORDER BY row_id;

Done.
1 rows affected.
2 rows affected.


row_id,principal_components,std_dev,proportion
1,"[-0.707106781186547, -5.55111512312578e-17, 0.707106781186547]",1.41421356237,0.857142857142
2,"[-5.55111512312578e-17, -1.0, -4.16333634234434e-17]",0.57735026919,0.142857142857


# 4.  Grouping with dense matrices
Create sample data in dense matrix form with a grouping column:

In [287]:
%%sql 
DROP TABLE IF EXISTS mat_group;

CREATE TABLE mat_group (
    id integer,
    row_vec double precision[],
    matrix_id integer
);

INSERT INTO mat_group VALUES
(1, '{1,2,3}', 1),
(2, '{2,1,2}', 1),
(3, '{3,2,1}', 1),
(4, '{1,2,3,4,5}' ,2),
(5, '{2,5,2,4,1}' ,2),
(6, '{5,4,3,2,1}' ,2);

SELECT * FROM mat_group ORDER BY id;

Done.
Done.
6 rows affected.
6 rows affected.


id,row_vec,matrix_id
1,"[1.0, 2.0, 3.0]",1
2,"[2.0, 1.0, 2.0]",1
3,"[3.0, 2.0, 1.0]",1
4,"[1.0, 2.0, 3.0, 4.0, 5.0]",2
5,"[2.0, 5.0, 2.0, 4.0, 1.0]",2
6,"[5.0, 4.0, 3.0, 2.0, 1.0]",2


# 5.0  Run the PCA function with grouping for a specified proportion of variance

In [289]:
%%sql
DROP TABLE IF EXISTS result_table_group, result_table_group_mean;

SELECT madlib.pca_train('mat_group',             -- Source table
                        'result_table_group',    -- Output table
                        'id',                    -- Row id of source table
                         0.8,                    -- Proportion of variance
                        'matrix_id');            -- Grouping column

SELECT * FROM result_table_group ORDER BY matrix_id, row_id;

Done.
1 rows affected.
3 rows affected.


row_id,principal_components,std_dev,proportion,matrix_id
1,"[0.707106781186548, 0.0, -0.707106781186547]",1.41421356237,0.857142857142,1
1,"[-0.555378486712784, -0.388303582074091, 0.0442457354870796, 0.255566375612852, 0.688115693174023]",3.23152203117,0.764102534485,2
2,"[0.587384101786277, -0.485138064894743, 0.311532046315153, -0.449458074050715, 0.347212037159181]",1.79553112719,0.235897465516,2


# 6.  Sparse matrices
Create sample data in sparse matrix form: 

In [266]:
%%sql
DROP TABLE IF EXISTS mat_sparse;

CREATE TABLE mat_sparse (
    row_id integer,
    col_id integer,
    value double precision
);

INSERT INTO mat_sparse VALUES
(1, 1, 1.0),
(2, 2, 2.0),
(3, 3, 3.0),
(4, 4, 4.0),
(1, 5, 5.0),
(2, 4, 6.0),
(3, 2, 7.0),
(4, 3, 8.0);

SELECT * FROM mat_sparse ORDER BY row_id, col_id;

Done.
Done.
8 rows affected.
8 rows affected.


row_id,col_id,value
1,1,1.0
1,5,5.0
2,2,2.0
2,4,6.0
3,2,7.0
3,3,3.0
4,3,8.0
4,4,4.0


This is what the sparse matrix looks like when put in dense form: 

In [290]:
%%sql
DROP TABLE IF EXISTS mat_dense;

SELECT madlib.matrix_densify('mat_sparse', 
                            'row=row_id, col=col_id, val=value', 
                            'mat_dense');

SELECT * FROM mat_dense ORDER BY row_id;

Done.
1 rows affected.
4 rows affected.


row_id,value
1,"[1.0, 0.0, 0.0, 0.0, 5.0]"
2,"[0.0, 2.0, 0.0, 6.0, 0.0]"
3,"[0.0, 7.0, 3.0, 0.0, 0.0]"
4,"[0.0, 0.0, 8.0, 4.0, 0.0]"


# 7.  Run the PCA sparse function for a specified number of principal components

In [291]:
%%sql
DROP TABLE IF EXISTS result_table, result_table_mean;

SELECT madlib.pca_sparse_train( 'mat_sparse',       -- Source table
                                'result_table',     -- Output table
                                'row_id',           -- Row id of source table
                                'col_id',           -- Column id of source table
                                'value',            -- Value of matrix at row_id, col_id
                                4,                  -- Actual number of rows in the matrix
                                5,                  -- Actual number of columns in the matrix
                                3);                 -- Number of principle components
                               
SELECT * FROM result_table ORDER BY row_id;

Done.
1 rows affected.
3 rows affected.


row_id,principal_components,std_dev,proportion
1,"[-0.0876046030186158, -0.0968983772909994, 0.798223884258747, 0.392314423461549, -0.438023015093079]",4.2136280383,0.436590030617
2,"[-0.0647272661608605, 0.877639526308692, 0.0872109579144954, -0.33647142089509, -0.323636330804303]",3.68408023747,0.333748701545
3,"[-0.0780380267884855, 0.177956517174911, -0.550872495873674, 0.711711105037595, -0.390190133942428]",3.0560690806,0.229661267838


# 8. Grouping with sparse matrices
Create sample data in sparse matrix form:

In [294]:
%%sql
DROP TABLE IF EXISTS mat_sparse_group;

CREATE TABLE mat_sparse_group (
    row_id integer,
    col_id integer,
    value double precision,
    matrix_id integer);

INSERT INTO mat_sparse_group VALUES
(1, 1, 1.0, 1),
(2, 2, 2.0, 1),
(3, 3, 3.0, 1),
(4, 4, 4.0, 1),
(1, 5, 5.0, 1),
(2, 4, 6.0, 2),
(3, 2, 7.0, 2),
(4, 3, 8.0, 2);

SELECT * FROM mat_sparse_group ORDER BY matrix_id, row_id, col_id;

Done.
Done.
8 rows affected.
8 rows affected.


row_id,col_id,value,matrix_id
1,1,1.0,1
1,5,5.0,1
2,2,2.0,1
3,3,3.0,1
4,4,4.0,1
2,4,6.0,2
3,2,7.0,2
4,3,8.0,2


Run the PCA function with grouping for a specified proportion of variance

In [295]:
%%sql
DROP TABLE IF EXISTS result_table_group, result_table_group_mean;

SELECT madlib.pca_sparse_train( 'mat_sparse_group',       -- Source table
                                'result_table_group',     -- Output table
                                'row_id',           -- Row id of source table
                                'col_id',           -- Column id of source table
                                'value',            -- Value of matrix at row_id, col_id
                                4,                 -- Actual number of rows in the matrix
                                5,                 -- Actual number of columns in the matrix
                                0.8,                 -- Proportion of variance
                                'matrix_id');
                               
SELECT * FROM result_table_group ORDER BY matrix_id, row_id;

Done.
1 rows affected.
4 rows affected.


row_id,principal_components,std_dev,proportion,matrix_id
1,"[-0.17805696611353, 0.0681313257646983, 0.140140679619953, 0.389110973431454, -0.890284830567652]",2.73659933166,0.544652792875,1
2,"[-0.0492086814863993, 0.149371585357526, 0.523758898591052, -0.800253642890858, -0.246043407431996]",2.06058314533,0.308800210824,1
1,"[0.0, -0.479486114660443, 0.860935716335886, -0.169949281213684, 0.0]",4.40325305088,0.520500333693,2
2,"[0.0, 0.689230898585949, 0.249586247854403, -0.680196643123417, 0.0]",3.74355664586,0.376220573443,2
